# Preparation table of revisions, users for restaurants.
In the review table, we will keep the restaurant reviews. We will use VADER to rate the sentiment reflected in each review for each restaurant, leading us to the base business table, summary statistics of the set of reviews. As we define our model, we will try to predict the stars awarded by a user, given the sentiment of the review he writes, the positioning of the business in terms of its characteristics and degree of specialization.

Other tables such as users, tips and check-in are prepared in case other applications are developed.



In [ ]:
import pandas as pd
import numpy as np
import json
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import datetime


In [ ]:
# Install and import nltk
!pip install nltk
import nltk

# Download the lexicon
nltk.download("vader_lexicon")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
from psutil import virtual_memory
print(f'Working with {round(virtual_memory().total / 1e9,2)} gigas of RAM.')


Working with 27.33 gigas of RAM.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.chdir('/content/drive/MyDrive/yelp/notebooks')

In [ ]:
# Read restaurant frame.
data_bus = pd.read_parquet(r'../data/restaurants/restaurants_open_definitive.parquet')

In [ ]:
data_bus.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'metro_area', 'category_1', 'category_2', 'category_3', 'category_4',
       'category_5', 'category_6', 'category_7', 'category_8', 'category_9',
       'category_10', 'category_11', 'category_12', 'Alcohol', 'BikeParking',
       'BusinessAcceptsBitcoin', 'BusinessAcceptsCreditCards',
       'BusinessParking', 'ByAppointmentOnly', 'Caters', 'DogsAllowed',
       'DriveThru', 'GoodForKids', 'HappyHour', 'HasTV', 'Music', 'NoiseLevel',
       'OutdoorSeating', 'RestaurantsAttire', 'RestaurantsDelivery',
       'RestaurantsGoodForGroups', 'RestaurantsPriceRange2',
       'RestaurantsReservations', 'RestaurantsTableService',
       'RestaurantsTakeOut', 'WheelchairAccessible', 'WiFi', 'romantic',
       'intimate', 'touristy', 'hipster', 'divey', 'classy', 'trendy',
       'upscale', 'casual', 'dessert', 'latenight', 'lunch', 'dinner',
      

In [ ]:
categories_columns = ['category_1', 'category_2', 'category_3', 'category_4', 'category_5', 
                             'category_6', 'category_7', 'category_8', 'category_9', 'category_10','category_11','category_12']

In [ ]:
data_bus[categories_columns].describe().T

,count,mean,std,min,25%,50%,75%,max
category_1,34950.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
category_2,34950.0,35.823433,369.555899,1.0,1.0,11.0,16.0,9999.0
category_3,34950.0,1777.585408,3779.722522,2.0,11.0,18.0,100.0,9999.0
category_4,34950.0,3281.498169,4647.666497,3.0,18.0,69.0,9999.0,9999.0
category_5,34950.0,5016.915107,4951.275219,5.0,44.0,777.5,9999.0,9999.0
category_6,34950.0,6785.945980,4624.153258,8.0,167.0,9999.0,9999.0,9999.0
category_7,34950.0,8045.254278,3925.854118,11.0,9999.0,9999.0,9999.0,9999.0
category_8,34950.0,8844.200143,3162.699030,14.0,9999.0,9999.0,9999.0,9999.0
category_9,34950.0,9327.568383,2473.299093,18.0,9999.0,9999.0,9999.0,9999.0
category_10,34950.0,9625.186753,1872.586409,22.0,9999.0,9999.0,9999.0,9999.0


In [ ]:
# We generate a frame only with the ids of restaurants, which will allow us to merge with the table
# table, only the restaurant reviews.
resturants_id = data_bus[['business_id']]

## Review table.
The review table does not contain any columns that need to be transformed or contain nested jsons. Given its length, we filter the reviews we are interested in from restaurants. And we take the opportunity to do a VADER sentiment analysis (http://eegilbert.org/papers/icwsm14.vader.hutto.pdf), which we will keep in the review table, and from which we will transfer summary data to our restaurant table.



In [ ]:
# We read the revision table in chunks.
review_json_path = r"../data/yelp_academic_dataset_review.json"
size = 1000000
review = pd.read_json(review_json_path, lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                      chunksize=size)

In [ ]:

# Generate a list of chucks to be merged.
chunk_list = []
for chunk_review in review:
    # Delete unusable column
    chunk_review = chunk_review.drop(['review_id'], axis=1)
    # Rename column to avoid conflict with business
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
    # Inner merge with restaurant_id table to keep only restaurant reviews 
    chunk_merged = pd.merge(resturants_id, chunk_review, on='business_id', how='inner')
    # Show progress.
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)
# Concatenate all chunks in a dataframe.
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

539432 out of 1,000,000 related reviews
548285 out of 1,000,000 related reviews
523165 out of 1,000,000 related reviews
529572 out of 1,000,000 related reviews
546425 out of 1,000,000 related reviews
543077 out of 1,000,000 related reviews
532542 out of 1,000,000 related reviews


In [ ]:
# Save to disk.
df.to_parquet("../data/restaurants/restaurants_reviews.parquet",
              compression='gzip')

In [ ]:
del [[df,review]]
gc.collect()

30

## SENTIMENT ANALYSIS.
With the functions of the nltk library related to VADER, we are going to qualify the polarity of each review, and through it, we will qualify as positive any message with polarity greater than 0.5, negative those with polarity less than -0.5, and neutral the rest. This process is slow, about 9 hours for our restaurants. 

We decided to discard reviews prior to 2016. Businesses evolve quickly and reviews too far back in time we think are no longer useful.
We will look at two VADER tables, one with value for each review for each restaurant and one with data grouped by restaurant.

In [ ]:
# We read restaurant reviews.
data_rew=pd.read_parquet("../data/restaurants/restaurants_reviews.parquet")

In [ ]:
data_rew

,business_id,user_id,review_stars,useful,funny,cool,text,date
0,MTSW4McQd7CbVtyjqoe9mw,6_SpY41LIHZuIaiDs5FMKA,4,0,0,1,This is nice little Chinese bakery in the hear...,2014-05-26 01:09:53
1,MTSW4McQd7CbVtyjqoe9mw,tCXElwhzekJEH6QJe3xs7Q,4,3,1,2,This is the bakery I usually go to in Chinatow...,2013-10-05 15:19:06
2,MTSW4McQd7CbVtyjqoe9mw,WqfKtI-aGMmvbA9pPUxNQQ,5,0,0,0,"A delightful find in Chinatown! Very clean, an...",2013-10-25 01:34:57
3,MTSW4McQd7CbVtyjqoe9mw,3-1va0IQfK-9tUMzfHWfTA,5,5,0,5,I ordered a graduation cake for my niece and i...,2018-05-20 17:58:57
4,MTSW4McQd7CbVtyjqoe9mw,EouCKoDfzaVG0klEgdDvCQ,4,2,1,1,HK-STYLE MILK TEA: FOUR STARS\n\nNot quite su...,2013-10-25 02:31:35
...,...,...,...,...,...,...,...,...
3762493,2O2K6SXPWv56amqxCECd4w,eWz12w7dzYlfrGnhTQ82Fg,5,0,0,0,This is my favorite food truck! I only wish I ...,2019-07-14 14:25:35
3762494,2O2K6SXPWv56amqxCECd4w,lDyhGApbGZ0_BoeJzRQq7g,5,1,0,1,This food truck was stupid. Stupidly delicious...,2021-06-25 23:22:26
3762495,2O2K6SXPWv56amqxCECd4w,-TTJ75--0NEAjvFCOV7rBg,5,0,0,0,Bubba never disappoints i go to his fb page an...,2016-12-09 21:38:05
3762496,2O2K6SXPWv56amqxCECd4w,8yFNNU7UmQcfzmcTvzTlOA,1,0,0,0,The truck was invited to our office for a part...,2020-02-19 22:59:06


Vader (Valence Aware Dictionary and Sentiment Reasoner) is a sentiment analysis tool developed to evaluate the tone or sentiment of natural language text. It is specifically designed to evaluate tone in social networks, making it very useful for analysing online opinions or comments.

Vader uses a set of rules and keyword-based scores to evaluate the tone of text. For example, if a comment includes words such as "happy", "content" or "excited", Vader will assign a positive score to that comment. Similarly, if a comment includes words such as "sad", "angry" or "upset", Vader will assign a negative score.

Vader also takes into account the context in which words are used and uses punctuation symbols and emojis to adjust the score. For example, if a comment includes an exclamation mark (!) after a positive word, Vader will assign an even more positive score to that comment.

Once Vader has assigned scores to all keywords in a comment, it sums the scores and assigns a final score to the comment. This final score can be used to determine whether the overall comment is positive, negative or neutral.



In [ ]:
# Import the lexicon 
from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [ ]:
# We define functions to apply to the dataframe.
def varder_polarity(text):
    return SentimentIntensityAnalyzer().polarity_scores(text)

def varder_analysis(score):
    if score['compound'] >= 0.05:
        return 'Positive'
    elif score['compound'] <= -0.5:
        return 'Negative'
    else:
        return 'Neutral'
data_rew['polarity'] = data_rew['text'].apply(varder_polarity)
data_rew['classification'] = data_rew['polarity'].apply(varder_analysis)

In [ ]:
vander_rev = pd.read_parquet("../data/restaurants/restaurants_reviews.parquet",columns=['business_id','review_stars','useful','funny','cool','date','polarity','classification'])

In [ ]:
# We generate a column for each result field of vaderplarity, as it returns a dictionary.
vander_rev = pd.concat([vander_rev.drop(['polarity'], axis=1), vander_rev['polarity'].apply(pd.Series)], axis=1)

In [ ]:
# Saving to file
vander_rev.to_parquet("../data/restaurants/restaurants_vander.parquet",
              compression = 'gzip')

In [ ]:
vander_rev = pd.read_parquet("../data/restaurants/restaurants_vander.parquet")

In [ ]:
vander_rev

,business_id,review_stars,useful,funny,cool,date,classification,compound,neg,neu,pos
0,MTSW4McQd7CbVtyjqoe9mw,4,0,0,1,2014-05-26 01:09:53,Positive,0.9390,0.043,0.723,0.234
1,MTSW4McQd7CbVtyjqoe9mw,4,3,1,2,2013-10-05 15:19:06,Positive,0.8905,0.000,0.903,0.097
2,MTSW4McQd7CbVtyjqoe9mw,5,0,0,0,2013-10-25 01:34:57,Positive,0.9770,0.000,0.539,0.461
3,MTSW4McQd7CbVtyjqoe9mw,5,5,0,5,2018-05-20 17:58:57,Positive,0.8828,0.000,0.709,0.291
4,MTSW4McQd7CbVtyjqoe9mw,4,2,1,1,2013-10-25 02:31:35,Negative,-0.7212,0.067,0.898,0.035
...,...,...,...,...,...,...,...,...,...,...,...
3762493,2O2K6SXPWv56amqxCECd4w,5,0,0,0,2019-07-14 14:25:35,Positive,0.9771,0.035,0.752,0.213
3762494,2O2K6SXPWv56amqxCECd4w,5,1,0,1,2021-06-25 23:22:26,Positive,0.7717,0.101,0.751,0.147
3762495,2O2K6SXPWv56amqxCECd4w,5,0,0,0,2016-12-09 21:38:05,Positive,0.8585,0.000,0.835,0.165
3762496,2O2K6SXPWv56amqxCECd4w,1,0,0,0,2020-02-19 22:59:06,Neutral,-0.4902,0.121,0.804,0.075


In [ ]:
# We transform the date column into date.
vander_rev.date = pd.to_datetime(vander_rev.date)

In [ ]:
# We generate our summary frame by business. Reviews after 31-12-2015, with review count, average star rating,
# compound average (composite sentiment), and minimum and maximum date.
vander_sum = vander_rev[vander_rev['date']> pd.Timestamp(2015,12,31)].groupby('business_id').agg({'review_stars':['count','mean'],'compound':'mean','date':['min','max']}).droplevel(0,axis=1)

In [ ]:
vander_sum.columns

Index(['count', 'mean', 'mean', 'min', 'max'], dtype='object')

In [ ]:
# We name the columns
vander_sum.columns = ['rev_counts','star_mean','vander_comp_mean','date_min','date_max']

In [ ]:
# We erase memory frame and clean rubbish.
del [[vander_rev]]
gc.collect()

72

## User tables.
We keep only a table of unique restaurant review users, and keep it on disk. We filter the users file to keep only the restaurant users data.

In [ ]:
# We keep the first revision user_id for each unique user
review_user_rest=data_rew[['user_id']].drop_duplicates(keep='first')

In [ ]:
review_user_rest.to_parquet("../data/restaurants/restaurants_revisadores.parquet",
              compression='gzip')

In [ ]:
# We free up memory.
del [[data_rew,review_user_rest]]
gc.collect()

8

In [ ]:
data_users = pd.read_parquet(r"../data/restaurants/restaurants_revisadores.parquet")

In [ ]:
data_users

,user_id
0,6_SpY41LIHZuIaiDs5FMKA
1,tCXElwhzekJEH6QJe3xs7Q
2,WqfKtI-aGMmvbA9pPUxNQQ
3,3-1va0IQfK-9tUMzfHWfTA
4,EouCKoDfzaVG0klEgdDvCQ
...,...
3762474,7Ka3Smzb7hlHi8gk0wByjw
3762481,B7TD5yTemGv50y4wM2EVNA
3762485,Bhv1MwgDREC5m9Tg2vFOMg
3762491,jlRf1i1xyeELscV-PG0SnQ


In [ ]:

users_json_path = r"/../data/yelp_academic_dataset_user.json"
size = 100000
users = pd.read_json(users_json_path, lines=True,
chunksize = size)

In [ ]:

# Vamos a generar una lista de chucks para despues fusionar.
chunk_list = []
chunk_split_list = []
for chunk_users in users:
   

    # Inner fusion con la tabla restaurant_id para quedarnos solo con los usuarios de restaurantes 
    chunk_merged = pd.merge(data_users, chunk_users, on='user_id', how='inner')
    
    # Mostrar progreso.
    print(f"{chunk_merged.shape[0]} out of {size:,} related users")
    chunk_list.append(chunk_merged)
    
# Concatenamos todos los trozos en un dataframe.
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)


87569 out of 100,000 related users
80483 out of 100,000 related users
80205 out of 100,000 related users
70739 out of 100,000 related users
77581 out of 100,000 related users
70214 out of 100,000 related users
72381 out of 100,000 related users
64293 out of 100,000 related users
67575 out of 100,000 related users
60269 out of 100,000 related users
62814 out of 100,000 related users
55872 out of 100,000 related users
59825 out of 100,000 related users
54236 out of 100,000 related users
57955 out of 100,000 related users
52509 out of 100,000 related users
56836 out of 100,000 related users
50599 out of 100,000 related users
51765 out of 100,000 related users
41639 out of 100,000 related users


In [ ]:
df.shape

(1275359, 22)

In [ ]:
df.columns

Index(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos'],
      dtype='object')

In [ ]:
df.to_parquet(r"../data/restaurants/restaurants_users.parquet",
              compression='gzip')

In [ ]:
del df
gc.collect()

65

##Tips and Check-in tables.

We generate restaurant tables of Tips and Check-ins. We transform check-in table.

In [ ]:
tips_json_path = r"../data/yelp_academic_dataset_tip.json"
tips_bus = pd.read_json(tips_json_path, lines=True)

In [ ]:
id_bus = pd.read_parquet(r"../data/restaurants/restaurants_id.parquet")

In [ ]:
tips_restaurants = pd.merge(tips_bus, id_bus, on ='business_id', how = 'inner')

In [ ]:
data_users = pd.read_parquet(r"../data/restaurants/restaurants_revisadores.parquet")

In [ ]:
tips_restaurants = pd.merge(tips_restaurants, data_users, on = 'user_id', how = 'inner')

In [ ]:
tips_restaurants.to_parquet("../data/restaurants/restaurants_tips.parquet",
              compression ='gzip')

In [ ]:
del tips_restaurants
gc.collect()

24

In [ ]:
checkin_json_path = r"/../data/yelp_academic_dataset_checkin.json"
checkin_bus = pd.read_json(checkin_json_path, lines=True)



In [ ]:
checkin_restaurants = pd.merge(checkin_bus, id_bus, on = 'business_id', how = 'inner')

In [ ]:
checkin_restaurants['date_list'] = checkin_restaurants['date'].apply(lambda row: row.split(','))

In [ ]:
checkin_restaurants['number_checkin'] = checkin_restaurants['date_list'].apply(lambda row: len(row))

In [ ]:
checkin_restaurants = checkin_restaurants.drop(columns = ['date'])

In [ ]:
checkin_restaurants.to_parquet("../data/restaurants/restaurants_cehck_in.parquet",
              compression='gzip')

In [ ]:
checkin_restaurants

,business_id,date_list,number_checkin
0,---kPU91CF4Lq2-WlRu9Lw,"[2020-03-13 21:10:56, 2020-06-02 22:18:06, 2...",11
1,--0iUa4sNDFiZFrAdIWhZQ,"[2010-09-13 21:43:09, 2011-05-04 23:08:15, 2...",10
2,--epgcb7xHGuJ-4PUeSLAw,"[2010-10-01 16:31:41, 2010-12-16 19:13:47, 2...",118
3,--hF_3v1JmU9nlu4zfXJ8Q,"[2021-09-15 16:08:07, 2021-09-30 17:10:48, 2...",7
4,--lqIzK-ZVTtgwiQM63XgQ,"[2011-07-29 16:37:02, 2011-10-24 16:00:29, 2...",21
...,...,...,...
34474,zzbZtgPYZS8sTIWQH6DwEw,"[2010-08-13 07:51:27, 2010-08-14 06:35:15, 2...",292
34475,zziDpuuJw-Km1J4BaGpBKA,"[2011-04-23 16:56:26, 2012-12-18 19:49:13, 2...",20
34476,zzjFdJwXuxBOGe9JeY_EMw,"[2015-01-14 19:21:48, 2015-06-26 17:16:54, 2...",27
34477,zznJox6-nmXlGYNWgTDwQQ,"[2013-03-23 16:22:47, 2013-04-07 02:03:12, 2...",67


In [ ]:
pd.DataFrame(checkin_restaurants['date_list'].to_list())

In [ ]:
resturants_id = pd.read_parquet("/../data/restaurants_id.parquet")


In [ ]:
checkin_restaurants = pd.read_parquet("/../data/restaurants_cehck_in.parquet")

In [ ]:
checkin_restaurants.shape

(34479, 3)

In [ ]:
dates_list = []
for i in range(0,34480, 800):
# expand df.tags into its own dataframe
  dates = checkin_restaurants['date_list'].iloc[i:i+800].apply(pd.Series)

  # rename each variable is tags
  dates = dates.rename(columns = lambda x : 'date_' + str(x))
  dates=dates.iloc[:, 0:2001]
  dates_list.append(dates)


In [ ]:
df = pd.concat(dates_list, ignore_index=True, join='outer', axis=0)

In [ ]:
df.shape

(34479, 2001)

In [ ]:
df

,date_0,date_1,date_2,date_3,date_4,date_5,date_6,date_7,date_8,date_9,...,date_1991,date_1992,date_1993,date_1994,date_1995,date_1996,date_1997,date_1998,date_1999,date_2000
0,2020-03-13 21:10:56,2020-06-02 22:18:06,2020-07-24 22:42:27,2020-10-24 21:36:13,2020-12-09 21:23:33,2021-01-20 17:34:57,2021-04-30 21:02:03,2021-05-25 21:16:54,2021-08-06 21:08:08,2021-10-02 15:15:42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-09-13 21:43:09,2011-05-04 23:08:15,2011-07-18 22:30:31,2012-09-07 20:28:50,2013-03-27 15:57:36,2013-08-13 00:31:34,2013-08-13 00:31:48,2013-09-23 17:39:38,2013-11-18 06:34:08,2014-04-12 23:04:47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-10-01 16:31:41,2010-12-16 19:13:47,2011-02-12 16:49:04,2011-09-17 13:28:47,2011-11-22 12:44:40,2011-12-21 14:54:56,2012-02-25 12:49:05,2012-03-05 15:08:24,2012-03-29 12:01:08,2012-06-03 13:58:14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-09-15 16:08:07,2021-09-30 17:10:48,2021-11-02 17:13:20,2021-11-02 17:14:41,2021-11-02 17:15:53,2021-12-08 18:21:10,2021-12-08 18:21:35,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-07-29 16:37:02,2011-10-24 16:00:29,2012-10-27 19:17:50,2013-05-13 22:03:28,2013-05-14 21:25:02,2013-05-28 22:18:02,2014-12-08 17:31:29,2015-03-02 18:41:47,2016-01-15 17:49:03,2016-03-06 16:55:09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34474,2010-08-13 07:51:27,2010-08-14 06:35:15,2010-08-22 05:15:11,2010-08-28 04:06:27,2010-09-05 06:16:09,2010-09-05 06:21:27,2010-09-11 06:53:33,2010-10-24 09:28:38,2011-01-08 05:14:52,2011-01-16 07:45:52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34475,2011-04-23 16:56:26,2012-12-18 19:49:13,2015-04-21 20:21:25,2015-05-10 16:43:31,2015-05-10 16:49:51,2015-10-03 18:53:50,2016-05-19 15:33:38,2016-07-09 17:39:03,2016-09-18 03:06:22,2016-11-19 19:51:25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34476,2015-01-14 19:21:48,2015-06-26 17:16:54,2015-10-29 20:44:37,2016-03-27 00:34:33,2016-06-03 15:55:16,2016-06-16 19:02:35,2016-06-17 16:12:43,2016-06-18 23:04:22,2016-10-25 22:54:08,2017-02-14 20:41:16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34477,2013-03-23 16:22:47,2013-04-07 02:03:12,2013-07-06 12:36:22,2014-04-14 22:48:44,2014-05-18 17:40:00,2014-05-24 18:56:26,2014-06-29 19:02:38,2014-08-16 13:03:15,2014-08-17 21:45:37,2014-08-18 11:02:53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.to_parquet("/../data/proyecto_dates_restaurants_cehck_in.parquet",
              compression='gzip')

In [ ]:
df['tag_2000'].isna().sum()

34106

In [ ]:
cols_2_extract = df.columns[0:2000]

df[cols_2_extract] = df[cols_2_extract].apply(lambda x : pd.to_datetime(x, errors = 'coerce'))

In [ ]:
df.to_parquet("/../data/proyecto_dates_restaurants_cehck_in.parquet",
              compression = 'gzip')

In [ ]:
check_in_dates = pd.read_parquet("../data/proyecto_dates_restaurants_cehck_in.parquet")

In [ ]:
check_in_dates = check_in_dates.fillna(pd.NaT)

In [ ]:
check_in_dates.date_2000 = pd.to_datetime(check_in_dates.date_2000 , errors='coerce')

In [ ]:
check_in_dates.dtypes

date_0       datetime64[ns]
date_1       datetime64[ns]
date_2       datetime64[ns]
date_3       datetime64[ns]
date_4       datetime64[ns]
                  ...      
date_1996    datetime64[ns]
date_1997    datetime64[ns]
date_1998    datetime64[ns]
date_1999    datetime64[ns]
date_2000    datetime64[ns]
Length: 2001, dtype: object

In [ ]:
for row in check_in_dates.index:
  try: 
    check_in_dates.iloc[row] = np.sort(check_in_dates.iloc[row].values,axis=1)
  except:
    pass


In [ ]:
checkin_restaurants = pd.concat([checkin_restaurants,check_in_dates],axis=1)

In [ ]:
checkin_restaurants = checkin_restaurants.drop(columns=['date_list'])

In [ ]:
df.to_parquet("../data/restautants_check_in_dates.parquet",
              compression='gzip')

In [ ]:
checkin_restaurants

,business_id,number_checkin,date_0,date_1,date_2,date_3,date_4,date_5,date_6,date_7,...,date_1991,date_1992,date_1993,date_1994,date_1995,date_1996,date_1997,date_1998,date_1999,date_2000
0,---kPU91CF4Lq2-WlRu9Lw,11,2020-03-13 21:10:56,2020-06-02 22:18:06,2020-07-24 22:42:27,2020-10-24 21:36:13,2020-12-09 21:23:33,2021-01-20 17:34:57,2021-04-30 21:02:03,2021-05-25 21:16:54,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,--0iUa4sNDFiZFrAdIWhZQ,10,2010-09-13 21:43:09,2011-05-04 23:08:15,2011-07-18 22:30:31,2012-09-07 20:28:50,2013-03-27 15:57:36,2013-08-13 00:31:34,2013-08-13 00:31:48,2013-09-23 17:39:38,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,--epgcb7xHGuJ-4PUeSLAw,118,2010-10-01 16:31:41,2010-12-16 19:13:47,2011-02-12 16:49:04,2011-09-17 13:28:47,2011-11-22 12:44:40,2011-12-21 14:54:56,2012-02-25 12:49:05,2012-03-05 15:08:24,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,--hF_3v1JmU9nlu4zfXJ8Q,7,2021-09-15 16:08:07,2021-09-30 17:10:48,2021-11-02 17:13:20,2021-11-02 17:14:41,2021-11-02 17:15:53,2021-12-08 18:21:10,2021-12-08 18:21:35,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,--lqIzK-ZVTtgwiQM63XgQ,21,2011-07-29 16:37:02,2011-10-24 16:00:29,2012-10-27 19:17:50,2013-05-13 22:03:28,2013-05-14 21:25:02,2013-05-28 22:18:02,2014-12-08 17:31:29,2015-03-02 18:41:47,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34474,zzbZtgPYZS8sTIWQH6DwEw,292,2010-08-13 07:51:27,2010-08-14 06:35:15,2010-08-22 05:15:11,2010-08-28 04:06:27,2010-09-05 06:16:09,2010-09-05 06:21:27,2010-09-11 06:53:33,2010-10-24 09:28:38,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
34475,zziDpuuJw-Km1J4BaGpBKA,20,2011-04-23 16:56:26,2012-12-18 19:49:13,2015-04-21 20:21:25,2015-05-10 16:43:31,2015-05-10 16:49:51,2015-10-03 18:53:50,2016-05-19 15:33:38,2016-07-09 17:39:03,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
34476,zzjFdJwXuxBOGe9JeY_EMw,27,2015-01-14 19:21:48,2015-06-26 17:16:54,2015-10-29 20:44:37,2016-03-27 00:34:33,2016-06-03 15:55:16,2016-06-16 19:02:35,2016-06-17 16:12:43,2016-06-18 23:04:22,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
34477,zznJox6-nmXlGYNWgTDwQQ,67,2013-03-23 16:22:47,2013-04-07 02:03:12,2013-07-06 12:36:22,2014-04-14 22:48:44,2014-05-18 17:40:00,2014-05-24 18:56:26,2014-06-29 19:02:38,2014-08-16 13:03:15,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
